In [2]:
import numpy as np
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE

In [16]:
# def tokenize_transcript(transcripts):
# 	for idx, song in enumerate(transcripts):
# 		lyrics = song["lyrics"]
# 		lyrics = lyrics.replace("\\n", " ").replace("[Hook", " ").replace("[Verse", " ").replace("b", " ", 1)
# 		transcripts[idx]["lyrics"] = re.findall(r"[a-z]+", lyrics.lower())

# 	return transcripts

with open("songData.json", "r") as f:
	song_transcripts = json.load(f)


songs = list(song_transcripts.values())
# songs = tokenize_transcript(songs)
# print(songs[0])

In [38]:
vectorizer = TfidfVectorizer(stop_words = "english")
songlist = [song["lyrics"] for song in songs]
songlist.append("go away")
lyric_matrix = vectorizer.fit_transform(songlist).transpose()
#print(lyric_matrix.shape)
#u, s, v = svds(lyric_matrix, k=100)
words_compressed, _, docs_compressed = svds(lyric_matrix, k = 40)
docs_compressed = docs_compressed.transpose()
print(docs_compressed.shape)

word_to_index = vectorizer.vocabulary_

index_to_word = {i:t for t,i, in word_to_index.items()}
words_compressed = normalize(words_compressed, axis =1)

(4545, 40)


In [39]:
#tsne = TSNE(verbose=1)
#projected_docs = tsne.fit_transform(docs_compressed)
#print(projected_docs.shape)
#plt.figure(figsize=(10,10))
#plt.scatter(projected_docs[:,0], projected_docs[:,1])
#plt.show()

docs_compressed = normalize(docs_compressed, axis=1)


def closest_songs(project_index_in, k=10):
	sims = docs_compressed.dot(docs_compressed[project_index_in,:])
	asort = np.argsort(-sims)[:k+1]
	return [(songs[i]["title"], sims[i]/sims[asort[0]]) for i in asort[1:]]
'''
for i in range(10):
	print(songs[i]["title"])
	for title, score in closest_songs(i):
		print("{}:{:.3f}".format(title[:40], score))
	print()
'''   


'\nfor i in range(10):\n\tprint(songs[i]["title"])\n\tfor title, score in closest_songs(i):\n\t\tprint("{}:{:.3f}".format(title[:40], score))\n\tprint()\n'

In [42]:
print(closest_songs(4544))

[(u'Wild Horses', 0.867114872124607), (u"Runnin'", 0.8555857166674825), (u'Slap', 0.8330602357600874), (u'Take Me Away', 0.8099121327029364), (u'Fly Away', 0.7602218306093043), (u'Up Up & Away', 0.7418489003260742), (u'Yesterday', 0.7038242336961213), (u'Beautiful Bliss', 0.6939507830039919), (u'One Call Away', 0.6906954190774769), (u'Paper, Scissors, Rock', 0.6858247271715243)]


In [15]:
query_matrix = vectorizer.fit_transform(["Good morning how are you doing today"]).transpose()
print(query_matrix.shape)
uq, sq, vq = svds(query_matrix, k=1)
q_compressed, _, qs_compressed = svds(query_matrix, k = 40)
qs_compressed = qs_compressed.transpose()
print(qs_compressed)

ValueError: Iterable over raw text documents expected, string object received.